In [1]:
import pandas as pd
import numpy as np

# Paths to the dataset files
train_file = '/content/cleaned_train.txt'
val_file = '/content/val.txt'
test_file = '/content/test.txt'

# Function to load the data from text files
def load_data(file_path):
    data = pd.read_csv(file_path, header=None, names=['sentence', 'label'], sep=';')
    return data

# Loading training, validation, and test datasets
train_data = load_data(train_file)
val_data = load_data(val_file)
test_data = load_data(test_file)

# Displaying the first few rows of each dataset to verify
train_data.head(), val_data.head(), test_data.head()


(                                            sentence    label
 0                            i didnt feel humiliated      joy
 1                              im feeling humiliated    anger
 2  i can go from feeling so hopeless to so damned...  sadness
 3   im grabbing a minute to post i feel greedy wrong    anger
 4  i am ever feeling nostalgic about the fireplac...     love,
                                             sentence    label
 0  im feeling quite sad and sorry for myself but ...  sadness
 1  i feel like i am still looking at a blank canv...  sadness
 2                     i feel like a faithful servant     love
 3                  i am just feeling cranky and blue    anger
 4  i can have for a treat or if i am feeling festive      joy,
                                             sentence    label
 0  im feeling rather rotten so im not very ambiti...  sadness
 1          im updating my blog because i feel shitty  sadness
 2  i never make her separate from me because i do...

In [2]:
from sklearn.preprocessing import LabelEncoder

# Label Encoding for converting emotion labels from strings to numerical categories
label_encoder = LabelEncoder()

# Preprocessing training, validation, and test datasets
train_data['label'] = label_encoder.fit_transform(train_data['label'])
val_data['label'] = label_encoder.transform(val_data['label'])
test_data['label'] = label_encoder.transform(test_data['label'])

# Extracting sentences and labels
train_sentences = train_data['sentence']
val_sentences = val_data['sentence']
test_sentences = test_data['sentence']

train_labels = train_data['label']
val_labels = val_data['label']
test_labels = test_data['label']

# Checking the transformation
train_data.head()


,sentence,label
0,i didnt feel humiliated,2
1,im feeling humiliated,0
2,i can go from feeling so hopeless to so damned...,4
3,im grabbing a minute to post i feel greedy wrong,0
4,i am ever feeling nostalgic about the fireplac...,3


In [3]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow as tf

# Setting up parameters for text vectorization
max_tokens = 10000
output_sequence_length = 100

# Text Vectorization
text_vectorizer = TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=output_sequence_length
)

# Adapt the text vectorizer to the training sentences
text_vectorizer.adapt(train_sentences)

# Vectorizing a sample to verify
sample_sentence = "I'm feeling happy today!"
vectorized_example = text_vectorizer([sample_sentence])

vectorized_example.numpy()


array([[ 16,   8, 154, 122,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [4]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(1,), dtype=tf.string),
    text_vectorizer,
    layers.Embedding(input_dim=max_tokens + 1, output_dim=128, input_length=output_sequence_length),
    layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    layers.GlobalMaxPooling1D(),
    layers.Dense(6, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_history = model.fit(
    train_sentences,
    train_labels,
    epochs=10,
    validation_data=(val_sentences, val_labels)
)


Epoch 1/10
500/500 [==============================] - 19s 36ms/step - loss: 0.8060 - accuracy: 0.7073 - val_loss: 0.1853 - val_accuracy: 0.9215
Epoch 2/10
500/500 [==============================] - 18s 35ms/step - loss: 0.1472 - accuracy: 0.9359 - val_loss: 0.1649 - val_accuracy: 0.9320
Epoch 3/10
500/500 [==============================] - 18s 35ms/step - loss: 0.0924 - accuracy: 0.9627 - val_loss: 0.1697 - val_accuracy: 0.9265
Epoch 4/10
500/500 [==============================] - 19s 38ms/step - loss: 0.0559 - accuracy: 0.9804 - val_loss: 0.1856 - val_accuracy: 0.9240
Epoch 5/10
500/500 [==============================] - 17s 34ms/step - loss: 0.0360 - accuracy: 0.9875 - val_loss: 0.2046 - val_accuracy: 0.9235
Epoch 6/10
500/500 [==============================] - 17s 34ms/step - loss: 0.0231 - accuracy: 0.9936 - val_loss: 0.2209 - val_accuracy: 0.9230
Epoch 7/10
500/500 [==============================] - 17s 34ms/step - loss: 0.0148 - accuracy: 0.9961 - val_loss: 0.2387 - val_accuracy:

In [5]:
test_loss, test_accuracy = model.evaluate(test_sentences, test_labels)


63/63 [==============================] - 0s 6ms/step - loss: 0.2792 - accuracy: 0.9180


In [9]:
new_text = ["my friend betrays me"]

prediction = model.predict(new_text)
predicted_label_index = np.argmax(prediction, axis=1)

predicted_label = label_encoder.inverse_transform(predicted_label_index)

# Print the predicted emotion
print(f"Predicted Emotion: {predicted_label[0]}")

1/1 [==============================] - 0s 98ms/step
Predicted Emotion: sadness
